In [ ]:
!pip install -U -q google-generativeai

In [ ]:
import google.generativeai as genai

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
!pip install pytube

In [ ]:
from pytube import YouTube

# YouTube video URL
url = "https://www.youtube.com/watch?v=lJXaNYTVjrQ&t=2s&ab_channel=VincentBossel"

# Create a YouTube object
yt = YouTube(url)

# Get the highest resolution stream
stream = yt.streams.get_highest_resolution()

# Download the video to the current directory
video_file_name = stream.download()

print("Video downloaded successfully!")

Video downloaded successfully!


In [ ]:
#video_file_name = "/content/THE COOK _ Award-Winning Short Film.mp4"

In [ ]:
import cv2
import os
import shutil

# Create or cleanup existing extracted image frames directory.
FRAME_EXTRACTION_DIRECTORY = "/content/frames"
FRAME_PREFIX = "_frame"

def create_frame_output_dir(output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    else:
        shutil.rmtree(output_dir)
        os.makedirs(output_dir)

def extract_frame_from_video(video_file_path, interval_seconds=5):
    print(f"Extracting frames from {video_file_path} at {interval_seconds} second intervals. This might take a bit...")
    create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
    vidcap = cv2.VideoCapture(video_file_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frame_duration = 1 / fps  # Time duration of each frame (in seconds)
    target_time = 0
    frame_count = 0
    while vidcap.isOpened():
        success, frame = vidcap.read()
        if not success:  # End of video
            break
        current_time = frame_count * frame_duration
        if current_time >= target_time:
            min = int(current_time // 60)
            sec = int(current_time % 60)
            time_string = f"{min:02d}:{sec:02d}"
            image_name = f"{FRAME_PREFIX}{time_string}.jpg"
            output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
            cv2.imwrite(output_filename, frame)
            target_time += interval_seconds
        frame_count += 1
    vidcap.release()  # Release the capture object
    print(f"Completed video frame extraction!\n\nExtracted: {frame_count} frames")

# Example usage:
video_file_name = stream.download()
extract_frame_from_video(video_file_name, interval_seconds=15)


Extracting frames from /content/THE COOK  Award-Winning Short Film.mp4 at 15 second intervals. This might take a bit...
Completed video frame extraction!

Extracted: 5101 frames


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:

      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

# Process each frame in the output directory
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)
files_to_upload = []
for file in files:
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

# Upload the files to the API
# Only upload a 10 second slice of files to reduce upload time.
# Change full_video to True to upload the whole video.
full_video = False

uploaded_files = []
print(f'Uploading {len(files_to_upload) if full_video else 10} files. This might take a bit...')

for file in files_to_upload if full_video else files_to_upload[1:10]:
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)

print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")

Uploading 10 files. This might take a bit...
Uploading: /content/frames/_frame00:15.jpg...
Uploading: /content/frames/_frame00:30.jpg...
Uploading: /content/frames/_frame00:45.jpg...
Uploading: /content/frames/_frame01:00.jpg...
Uploading: /content/frames/_frame01:15.jpg...
Uploading: /content/frames/_frame01:30.jpg...
Uploading: /content/frames/_frame01:45.jpg...
Uploading: /content/frames/_frame02:00.jpg...
Uploading: /content/frames/_frame02:15.jpg...
Completed file uploads!

Uploaded: 9 files


In [ ]:
# Create the prompt.
prompt = "describe what you see"

# Set the model to Gemini 1.5 Pro.
#model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

system_instruction = "you are a movie watcher which inspects movies to find scenes that people want to skip and find \nyou will be given a link and a description of what is unwanted to be seen and you will watch return the timestamps where these things happen and when it ends so people can find and skip those scenes"

model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

# Make GenerateContent request with the structure described above.
def make_request(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

# Make the LLM request.
request = make_request(prompt, uploaded_files)
response = model.generate_content(request,
                                  request_options={"timeout": 12000})
print(response.text)

## Scene Breakdown:

The provided screenshots depict a sequence involving food preparation and consumption, with potential unsettling or disturbing elements.  Here's a breakdown:

**00:15 - 00:30:** A chef is working in a dimly lit kitchen, the atmosphere seems tense and serious. 
**00:30 - 01:00:** Focus on food preparation, but the specific ingredients and methods are unclear, potentially hinting at unusual or unsettling culinary practices.
**01:00 - 01:15:** The chef's intense focus on a seemingly empty bowl might suggest unconventional or disturbing elements within the dish.
**01:15 - 01:30:** A dark scene with blurred lights, creating suspense and anticipation for something ominous to happen.
**01:30 - 02:00:** A covered dish is presented, maintaining the mystery around the food. The reveal shows a minimalist dish with unusual elements like a flower and what appears to be a cigarette, further hinting at the unsettling nature of the meal.
**02:00 - 02:15:** A man consuming the ciga

In [ ]:
print(f'Deleting {len(uploaded_files)} images. This might take a bit...')
for file in uploaded_files:
  genai.delete_file(file.response.name)
  print(f'Deleted {file.file_path} at URI {file.response.uri}')
print(f"Completed deleting files!\n\nDeleted: {len(uploaded_files)} files")

Deleting 9 images. This might take a bit...
Deleted /content/frames/_frame00:15.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/xfw769wl799p
Deleted /content/frames/_frame00:30.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/c5ds84cdkdv3
Deleted /content/frames/_frame00:45.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/4blj8mgyxisn
Deleted /content/frames/_frame01:00.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/n35lyd6b0tr3
Deleted /content/frames/_frame01:15.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/1n25p3t7nfm2
Deleted /content/frames/_frame01:30.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/v5dnp3nwgsxm
Deleted /content/frames/_frame01:45.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/ycvm8ouc4kr7
Deleted /content/frames/_frame02:00.jpg at URI https://generativelanguage.googleapis.com/v1beta/files/jd50kfm55js6
Deleted /content/frames/_frame02:15.